## 库导入

In [1]:
import pandas as pd # 用于处理数据的工具
import lightgbm as lgb # 机器学习模型 LightGBM
from sklearn.metrics import mean_absolute_error # 评分 MAE 的计算函数
from sklearn.model_selection import train_test_split # 拆分训练集与验证集工具
from tqdm import tqdm # 显示循环的进度条工具

## 数据准备

In [2]:
train_dataset = pd.read_csv("./data/train.csv") # 原始训练数据。
test_dataset = pd.read_csv("./data/test.csv") # 原始测试数据（用于提交）。

submit = pd.DataFrame() # 定义提交的最终数据。
submit["序号"] = test_dataset["序号"] # 对齐测试数据的序号。

MAE_scores = dict() # 定义评分项。

In [3]:
pred_labels = list(train_dataset.columns[-34:]) # 需要预测的标签。
train_set, valid_set = train_test_split(train_dataset, test_size=0.2) # 拆分数据集。

# 设定 LightGBM 训练参，查阅参数意义：https://lightgbm.readthedocs.io/en/latest/Parameters.html
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'min_child_weight': 5,
        'num_leaves': 2 ** 5,
        'lambda_l2': 10,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 4,
        'learning_rate': 0.05,
        'seed': 2023,
        'nthread' : 16,
        'verbose' : -1,
    }

no_info = lgb.callback.log_evaluation(period=-1) # 禁用训练日志输出。

## 时间特征函数

In [4]:
def time_feature(data: pd.DataFrame, pred_labels: list=None) -> pd.DataFrame:
    """提取数据中的时间特征。

    输入: 
        data: Pandas.DataFrame
            需要提取时间特征的数据。

        pred_labels: list, 默认值: None
            需要预测的标签的列表。如果是测试集，不需要填入。
    
    输出: data: Pandas.DataFrame
            提取时间特征后的数据。
    """
    
    data = data.copy() # 复制数据，避免后续影响原始数据。
    data = data.drop(columns=["序号"]) # 去掉”序号“特征。
    
    data["时间"] = pd.to_datetime(data["时间"]) # 将”时间“特征的文本内容转换为 Pandas 可处理的格式。
    data["month"] = data["时间"].dt.month # 添加新特征“month”，代表”当前月份“。
    data["day"] = data["时间"].dt.day # 添加新特征“day”，代表”当前日期“。
    data["hour"] = data["时间"].dt.hour # 添加新特征“hour”，代表”当前小时“。
    data["minute"] = data["时间"].dt.minute # 添加新特征“minute”，代表”当前分钟“。
    data["weekofyear"] = data["时间"].dt.isocalendar().week.astype(int) # 添加新特征“weekofyear”，代表”当年第几周“，并转换成 int，否则 LightGBM 无法处理。
    data["dayofyear"] = data["时间"].dt.dayofyear # 添加新特征“dayofyear”，代表”当年第几日“。
    data["dayofweek"] = data["时间"].dt.dayofweek # 添加新特征“dayofweek”，代表”当周第几日“。
    data["is_weekend"] = data["时间"].dt.dayofweek // 6 # 添加新特征“is_weekend”，代表”是否是周末“，1 代表是周末，0 代表不是周末。

    data = data.drop(columns=["时间"]) # LightGBM 无法处理这个特征，它已体现在其他特征中，故丢弃。

    if pred_labels: # 如果提供了 pred_labels 参数，则执行该代码块。
        data = data.drop(columns=[*pred_labels]) # 去掉所有待预测的标签。
    
    return data # 返回最后处理的数据。

## 模型训练

In [5]:
test_features = time_feature(test_dataset) # 处理测试集的时间特征，无需 pred_labels。


# 从所有待预测特征中依次取出标签进行训练与预测。
for pred_label in tqdm(pred_labels):
    train_features = time_feature(train_set, pred_labels=pred_labels) # 处理训练集的时间特征。
    train_labels = train_set[pred_label] # 训练集的标签数据。
    train_data = lgb.Dataset(train_features, label=train_labels) # 将训练集转换为 LightGBM 可处理的类型。

    valid_features = time_feature(valid_set, pred_labels=pred_labels) # 处理验证集的时间特征。
    valid_labels = valid_set[pred_label] # 验证集的标签数据。
    valid_data = lgb.Dataset(valid_features, label=valid_labels) # 将验证集转换为 LightGBM 可处理的类型。

    # 训练模型，参数依次为：导入模型设定参数、导入训练集、设定模型迭代次数（200）、导入验证集、禁止输出日志
    model = lgb.train(lgb_params, train_data, 200, valid_sets=valid_data, callbacks=[no_info])

    valid_pred = model.predict(valid_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行验证集预测。
    test_pred = model.predict(test_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行测试集预测。
    MAE_score = mean_absolute_error(valid_pred, valid_labels) # 计算验证集预测数据与真实数据的 MAE。
    MAE_scores[pred_label] = MAE_score # 将对应标签的 MAE 值 存入评分项中。

    submit[pred_label] = test_pred # 将测试集预测数据存入最终提交数据中。
     
submit.to_csv('submit_resultV1.csv', index=False) # 保存最后的预测结果到 submit_result.csv。
print(MAE_scores) # 查看各项的 MAE 值。

100%|██████████| 34/34 [00:14<00:00,  2.31it/s]


{'上部温度1': 5.433900073709508, '上部温度2': 4.891030961048869, '上部温度3': 3.513313225131742, '上部温度4': 2.191398096628142, '上部温度5': 2.7729626562612077, '上部温度6': 2.2511004012458975, '上部温度7': 1.510942027512314, '上部温度8': 0.13189418767196318, '上部温度9': 0.06854791631632012, '上部温度10': 0.059335721952227145, '上部温度11': 0.08404001751708345, '上部温度12': 0.10777829641791382, '上部温度13': 0.10099665719207278, '上部温度14': 0.12480737652336503, '上部温度15': 0.21270456077506092, '上部温度16': 0.29091390638525577, '上部温度17': 0.9002334032843843, '下部温度1': 3.249149217959511, '下部温度2': 1.4292890673058718, '下部温度3': 0.6279842281928067, '下部温度4': 0.3223641092151853, '下部温度5': 1.0752196795851987, '下部温度6': 0.6681264660774292, '下部温度7': 0.3034945653049688, '下部温度8': 0.06933763064576512, '下部温度9': 6.130724797399228, '下部温度10': 0.06541844994754716, '下部温度11': 0.10301468157368411, '下部温度12': 0.12458255480581751, '下部温度13': 0.09633756312731848, '下部温度14': 0.11069924116616313, '下部温度15': 0.17795214934883957, '下部温度16': 0.24227430839299752, '下部温度17': 0.2745

实践了一遍datawhale团队提供的baseline,初步了解了如何就比赛提供的数据，通过baseline提供的分析流程，划分训练集和测试集，并根据数据的时间特征构建特征函数对数据进行数据处理，通过LightGBM模型进行模型训练，得到了预测的结果。

目前对baseline中的许多知识(LightGBM模型的特点；数据分割的方法；模型训练的细节等)还没有深入的了解，处于一个囫囵吞枣的状态。

下一步的打算是先把baseline尽可能搞的更清楚，同时加深对与赛题数据相适应的数据特征提取思路和方法以及经典机器学习算法的认识。从而得到对baseline进行优化的思路和方法。